# PROJECT

* **For a certain user **recommend 10 films** while using **item-based** and **user-based** recommender methods.**

![Alt text](recommender.jpg)

## USER BASED RECOMMENDER SYSTEM

This dataset is taken from **MovieLens** which has a service that recommend movies. This dataset has movies and the ratings for these movies. For 27.278 films there are 2.000.0263 ratings for the movies. This dataset is created in 17 SEptember 2016. It has 138.493 users between 09.01.1995 - 31.03.2015 dates. And every user has rated a film 20 times at least.

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [3]:
movies = pd.read_csv("./datasets/movie.csv")
ratings = pd.read_csv("./datasets/rating.csv")

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


## Task 1: Prepare Data

### Step 1: Merge rating and movie datasets

In [6]:
df = pd.merge(ratings,movies,how="left",on="movieId")

In [7]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


### Step 2:  Find and remove the movies which total vote number is less then 1000 after storing the name of the movies in a list.

In [8]:
df["title"].value_counts()

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
                                    ...  
Richard III (1912)                      1
The Nautical Chart (2007)               1
Two Arabian Knights (1927)              1
Night Flight (1933)                     1
Série noire (1979)                      1
Name: title, Length: 26729, dtype: int64

In [9]:
movies = pd.DataFrame(df["title"].value_counts())
movies.head()

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715


In [10]:
uncommon_movies = movies[movies["title"]<5000]
uncommon_movies.head()

,title
Big Trouble in Little China (1986),4999
It Could Happen to You (1994),4993
RoboCop 2 (1990),4975
Richie Rich (1994),4965
"Natural, The (1984)",4964


In [11]:
uncommon_movies.shape

(25724, 1)

In [12]:
rare_movies = movies[movies["title"]<10000].index
rare_movies

Index(['Ronin (1998)', 'My Best Friend's Wedding (1997)', 'Flintstones, The (1994)', 'Copycat (1995)', 'Space Jam (1996)', 'Harry Potter and the Goblet of Fire (2005)', 'Junior (1994)', 'Perfect Storm, The (2000)', 'Swingers (1996)', 'Fatal Attraction (1987)',
       ...
       'The Magnificent Trio (1966)', 'Five Weeks in a Balloon (1962)', 'The Lost Squadron (1932)', 'Under the Stars (Bajo las estrellas) (2007)', 'Payment Deferred (1932)', 'Richard III (1912)', 'The Nautical Chart (2007)', 'Two Arabian Knights (1927)', 'Night Flight (1933)', 'Série noire (1979)'],
      dtype='object', length=26267)

In [13]:
rare_movies.shape

(26267,)

In [14]:
common_movies = df[~df["title"].isin(rare_movies)]
common_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,1,112,3.5,2004-09-10 03:09:00,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime


In [15]:
df.shape

(20000263, 6)

In [16]:
common_movies.shape

(9050403, 6)

In [17]:
df["title"].nunique()

26729

In [18]:
common_movies["title"].nunique()

462

### Step 3: Make a pivot table that has user ids in rows and films in the columns with rating values.

In [19]:
common_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,1,112,3.5,2004-09-10 03:09:00,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime


In [20]:
user_movie_df = common_movies.pivot_table(index=["userId"],columns=["title"],values="rating")

In [21]:
user_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),"American President, The (1995)",Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Bad Boys (1995),Basic Instinct (1992),Batman (1989),Batman Begins (2005),Batman Forever (1995),Batman Returns (1992),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),Best in Show (2000),Beverly Hills Cop (1984),Beverly Hills Cop III (1994),Big (1988),Big Fish (2003),"Big Lebowski, The (1998)","Birdcage, The (1996)","Birds, The (1963)",Black Hawk Down (2001),Blade (1998),Blade Runner (1982),"Blair Witch Project, The (1999)",Blazing Saddles (1974),"Blues Brothers, The (1980)",Boogie Nights (1997),"Boot, Das (Boat, The) (1981)","Bourne Identity, The (2002)","Bourne Supremacy, The (2004)","Bourne Ultimatum, The (2007)",Bowling for Columbine (2002),Braveheart (1995),Brazil (1985),"Breakfast Club, The (1985)","Bridge on the River Kwai, The (1957)",Bridget Jones's Diary (2001),Broken Arrow (1996),"Bug's Life, A (1998)",Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Casablanca (1942),Casino (1995),Casino Royale (2006),Casper (1995),Cast Away (2000),Catch Me If You Can (2002),Charlie's Angels (2000),Chasing Amy (1997),Chicken Run (2000),Chinatown (1974),Chocolat (2000),"Christmas Story, A (1983)",Citizen Kane (1941),City Slickers II: The Legend of Curly's Gold (1994),City of God (Cidade de Deus) (2002),Clear and Present Danger (1994),Clerks (1994),"Client, The (1994)",Cliffhanger (1993),"Clockwork Orange, A (1971)",Close Encounters of the Third Kind (1977),Clueless (1995),Cocoon (1985),Con Air (1997),Coneheads (1993),Congo (1995),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Crimson Tide (1995),Crocodile Dundee (1986),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)","Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dark City (1998),"Dark Knight, The (2008)",Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),Deep Impact (1998),"Deer Hunter, The (1978)",Demolition Man (1993),"Departed, The (2006)",Desperado (1995),"Devil's Advocate, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Dirty Dancing (1987),Disclosure (1994),Dogma (1999),Donnie Brasco (1997),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dracula (Bram Stoker's Dracula) (1992),Dragonheart (1996),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),Edward Scissorhands (1990),Election (1999),Enemy of the State (1998),"English Patient, The (1996)",Eraser (1996),Erin Brockovich (2000),Eternal Sunshine of the Spotless Mind (2004),Executive Decision (1996),"Exorcist, The (1973)",Eyes Wide Shut (1999),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Ferris Bueller's Day Off (1986),"Few Good Men, A (1992)",Field of Dreams (1989),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)",First Knight (1995),"Fish Called Wanda, A (1988)"

### Step 4: Functonize steps

In [ ]:
def create_user_movie_df(min_rating_number=10000):
    movie_df = pd.read_csv("./datasets/movie.csv")
    rating_df = pd.read_csv("./datasets/rating.csv")
    df = pd.merge(rating_df,movie_df,how="left",on="movieId")
    movies = pd.DataFrame(df["title"].value_counts())
    rare_movies = movies[movies["title"]<min_rating_number].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"],columns=["title"],values="rating")
    return user_movie_df

new_df = create_user_movie_df()

## Task 2: Determination of Suggestions and Movies Watched by the User

### Step 1: Select a random user

In [24]:
user_movie_df = new_df

In [25]:
user_id = pd.Series(user_movie_df.index).sample(1).values[0]
user_id

127072

### Step 2:  Create a new datafarme called "**random_user_df**" that has observations for a certain chosen user.

In [26]:
random_user_df = user_movie_df[user_movie_df.index==user_id]
random_user_df.head(10)

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),"American President, The (1995)",Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Bad Boys (1995),Basic Instinct (1992),Batman (1989),Batman Begins (2005),Batman Forever (1995),Batman Returns (1992),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),Best in Show (2000),Beverly Hills Cop (1984),Beverly Hills Cop III (1994),Big (1988),Big Fish (2003),"Big Lebowski, The (1998)","Birdcage, The (1996)","Birds, The (1963)",Black Hawk Down (2001),Blade (1998),Blade Runner (1982),"Blair Witch Project, The (1999)",Blazing Saddles (1974),"Blues Brothers, The (1980)",Boogie Nights (1997),"Boot, Das (Boat, The) (1981)","Bourne Identity, The (2002)","Bourne Supremacy, The (2004)","Bourne Ultimatum, The (2007)",Bowling for Columbine (2002),Braveheart (1995),Brazil (1985),"Breakfast Club, The (1985)","Bridge on the River Kwai, The (1957)",Bridget Jones's Diary (2001),Broken Arrow (1996),"Bug's Life, A (1998)",Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Casablanca (1942),Casino (1995),Casino Royale (2006),Casper (1995),Cast Away (2000),Catch Me If You Can (2002),Charlie's Angels (2000),Chasing Amy (1997),Chicken Run (2000),Chinatown (1974),Chocolat (2000),"Christmas Story, A (1983)",Citizen Kane (1941),City Slickers II: The Legend of Curly's Gold (1994),City of God (Cidade de Deus) (2002),Clear and Present Danger (1994),Clerks (1994),"Client, The (1994)",Cliffhanger (1993),"Clockwork Orange, A (1971)",Close Encounters of the Third Kind (1977),Clueless (1995),Cocoon (1985),Con Air (1997),Coneheads (1993),Congo (1995),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Crimson Tide (1995),Crocodile Dundee (1986),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)","Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dark City (1998),"Dark Knight, The (2008)",Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),Deep Impact (1998),"Deer Hunter, The (1978)",Demolition Man (1993),"Departed, The (2006)",Desperado (1995),"Devil's Advocate, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Dirty Dancing (1987),Disclosure (1994),Dogma (1999),Donnie Brasco (1997),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dracula (Bram Stoker's Dracula) (1992),Dragonheart (1996),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),Edward Scissorhands (1990),Election (1999),Enemy of the State (1998),"English Patient, The (1996)",Eraser (1996),Erin Brockovich (2000),Eternal Sunshine of the Spotless Mind (2004),Executive Decision (1996),"Exorcist, The (1973)",Eyes Wide Shut (1999),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Ferris Bueller's Day Off (1986),"Few Good Men, A (1992)",Field of Dreams (1989),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)",First Knight (1995),"Fish Called Wanda, A (1988)"

### Step 3: Create a movies_watched list to add the movies that chosen user has voted.

In [27]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['Aladdin (1992)',
 'Austin Powers: International Man of Mystery (1997)',
 'Clueless (1995)',
 'Fargo (1996)',
 'Forrest Gump (1994)',
 'Full Monty, The (1997)',
 'Grosse Pointe Blank (1997)',
 'My Cousin Vinny (1992)',
 'Notting Hill (1999)',
 'Rain Man (1988)',
 'Saving Private Ryan (1998)',
 'Shakespeare in Love (1998)',
 'Sleepless in Seattle (1993)',
 'Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Truth About Cats & Dogs, The (1996)',
 'Wedding Singer, The (1998)',
 'While You Were Sleeping (1995)',
 "You've Got Mail (1998)"]

## Task 3: Reaching The Other Users' ids and Data that Watch the Same Films as the Chosen User

### Step 1: Select the movies that chosen user has watched from user_movie_df and create **movies_wathced_df**.

In [28]:
movies_wathced_df = user_movie_df[movies_watched]
movies_wathced_df.head()

title,Aladdin (1992),Austin Powers: International Man of Mystery (1997),Clueless (1995),Fargo (1996),Forrest Gump (1994),"Full Monty, The (1997)",Grosse Pointe Blank (1997),My Cousin Vinny (1992),Notting Hill (1999),Rain Man (1988),Saving Private Ryan (1998),Shakespeare in Love (1998),Sleepless in Seattle (1993),Toy Story (1995),Toy Story 2 (1999),"Truth About Cats & Dogs, The (1996)","Wedding Singer, The (1998)",While You Were Sleeping (1995),You've Got Mail (1998)
userId,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


### Step 2: For every user calculate the how much movies are the same movies that are watched with the selected user's movies

In [29]:
user_movie_count = movies_wathced_df.T.notnull().sum()
user_movie_count

userId
1         0
2         0
3         2
4         1
5         3
         ..
138489    0
138490    0
138491    1
138492    2
138493    7
Length: 137658, dtype: int64

In [30]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId","count"]
user_movie_count.head()

,userId,count
0,1,0
1,2,0
2,3,2
3,4,1
4,5,3


### Step 3: Step 3: Create a user_same_movies list that has user ids for users that voted 60 percent of the movies that the selected user has voted 


In [31]:
number = len(movies_watched)
filter = number*(60/100)
user_same_movies = user_movie_count[user_movie_count["count"] > filter]["userId"]
user_same_movies

23            24
53            54
57            58
90            91
123          124
           ...  
137552    138387
137569    138404
137575    138411
137618    138454
137620    138456
Name: userId, Length: 7356, dtype: int64

In [32]:
len(user_same_movies)

7356

## Task 4: Selecting the users that are very similar to user that will be recommended movies to.

### Step 1: Filter the **movies_watched_df** which has the users that is related to the selected user from the **user_same_movies** list.

In [33]:
movies_wathced_df.head()

title,Aladdin (1992),Austin Powers: International Man of Mystery (1997),Clueless (1995),Fargo (1996),Forrest Gump (1994),"Full Monty, The (1997)",Grosse Pointe Blank (1997),My Cousin Vinny (1992),Notting Hill (1999),Rain Man (1988),Saving Private Ryan (1998),Shakespeare in Love (1998),Sleepless in Seattle (1993),Toy Story (1995),Toy Story 2 (1999),"Truth About Cats & Dogs, The (1996)","Wedding Singer, The (1998)",While You Were Sleeping (1995),You've Got Mail (1998)
userId,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [34]:
final_df = movies_wathced_df[movies_wathced_df.index.isin(user_same_movies)]
final_df.head()

title,Aladdin (1992),Austin Powers: International Man of Mystery (1997),Clueless (1995),Fargo (1996),Forrest Gump (1994),"Full Monty, The (1997)",Grosse Pointe Blank (1997),My Cousin Vinny (1992),Notting Hill (1999),Rain Man (1988),Saving Private Ryan (1998),Shakespeare in Love (1998),Sleepless in Seattle (1993),Toy Story (1995),Toy Story 2 (1999),"Truth About Cats & Dogs, The (1996)","Wedding Singer, The (1998)",While You Were Sleeping (1995),You've Got Mail (1998)
userId,,,,,,,,,,,,,,,,,,,
24,4.0,5.0,3.0,2.0,5.0,3.0,3.0,3.0,2.0,NaN,5.0,4.0,4.0,4.0,3.0,NaN,4.0,3.0,NaN
54,NaN,4.0,2.0,5.0,5.0,4.0,NaN,NaN,4.0,NaN,5.0,2.0,4.0,4.0,5.0,3.0,2.0,4.0,3.0
58,NaN,4.0,NaN,5.0,4.5,4.5,NaN,4.0,5.0,4.0,5.0,4.5,5.0,5.0,4.5,NaN,NaN,NaN,NaN
91,NaN,4.0,NaN,3.5,4.0,4.0,NaN,3.5,4.0,3.5,4.0,3.5,3.5,4.0,4.5,2.0,3.5,2.5,2.5
124,NaN,3.0,2.0,5.0,4.0,5.0,NaN,NaN,5.0,5.0,2.0,NaN,3.5,5.0,5.0,NaN,1.0,NaN,4.5


In [35]:
final_df.shape

(7356, 19)

### Step 2: Create a corr_df dataframe that users correlation with each other.

In [36]:
corr_df = final_df.T.corr().unstack().sort_values()
corr_df.head()

userId  userId
107720  16120    -1.0
13494   101247   -1.0
109717  52335    -1.0
67319   120429   -1.0
79368   71198    -1.0
dtype: float64

In [37]:
cor_df = pd.DataFrame(corr_df,columns=["corr"])
cor_df.index.names = ["user_id_1","user_id_2"]
cor_df = cor_df.reset_index()
cor_df.head()

,user_id_1,user_id_2,corr
0,107720,16120,-1.0
1,13494,101247,-1.0
2,109717,52335,-1.0
3,67319,120429,-1.0
4,79368,71198,-1.0


In [38]:
cor_df[cor_df["user_id_1"]==user_id]

,user_id_1,user_id_2,corr
9496,127072,32454,-0.911685
15413,127072,16514,-0.894058
102231,127072,75962,-0.798893
139004,127072,75572,-0.777017
155094,127072,111088,-0.768800
...,...,...,...
53982665,127072,127072,1.000000
54099400,127072,1376,NaN
54099401,127072,7894,NaN
54099402,127072,48498,NaN


### Step 3: Create a new dataframe as **top_users** that has selected user and at least 0.65 correlation with the other users.


In [39]:
top_users = cor_df.loc[(cor_df["user_id_1"] == user_id) & cor_df["corr"] > 0.65,["user_id_2","corr"]]
top_users.head()

,user_id_2,corr
9496,32454,-0.911685
15413,16514,-0.894058
102231,75962,-0.798893
139004,75572,-0.777017
155094,111088,-0.768800


In [40]:
top_users.reset_index(inplace=True)
top_users.drop(["index"],inplace=True,axis=1)
top_users.head()

,user_id_2,corr
0,32454,-0.911685
1,16514,-0.894058
2,75962,-0.798893
3,75572,-0.777017
4,111088,-0.768800


In [41]:
top_users = top_users.sort_values(by="corr",ascending=False)
top_users.head()

,user_id_2,corr
7346,127072,1.000000
7345,53968,0.915702
7344,96649,0.879964
7343,74680,0.857493
7342,58572,0.842852


In [42]:
top_users.columns = ["userId","corr"]
top_users.head()

,userId,corr
7346,127072,1.000000
7345,53968,0.915702
7344,96649,0.879964
7343,74680,0.857493
7342,58572,0.842852


### Step 4: Merge top_users dataframe with rating dataset

In [43]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


* how="inner" gets the common data from two datasets it does not bring all of it.

In [44]:
top_users_ratings = top_users.merge(ratings,on="userId",how="inner")
top_users_ratings = top_users_ratings[top_users_ratings["userId"]!=user_id]
top_users_ratings.head()

,userId,corr,movieId,rating,timestamp
27,53968,0.915702,1,2.0,2000-07-01 15:16:29
28,53968,0.915702,4,3.0,2000-07-01 15:26:07
29,53968,0.915702,5,5.0,2000-07-01 15:22:55
30,53968,0.915702,6,5.0,2000-07-01 02:23:06
31,53968,0.915702,10,3.0,2000-07-01 02:39:06


## Task 5: Calculate Weighted Average Recommendation Score ve Save First 5 Movies

### Step 1: Create a **weighted_rating** column that has rating*corr for every user

In [45]:
top_users_ratings["weighted_rating"] = top_users_ratings["corr"]*top_users_ratings["rating"]
top_users_ratings.head()

,userId,corr,movieId,rating,timestamp,weighted_rating
27,53968,0.915702,1,2.0,2000-07-01 15:16:29,1.831403
28,53968,0.915702,4,3.0,2000-07-01 15:26:07,2.747105
29,53968,0.915702,5,5.0,2000-07-01 15:22:55,4.578508
30,53968,0.915702,6,5.0,2000-07-01 02:23:06,4.578508
31,53968,0.915702,10,3.0,2000-07-01 02:39:06,2.747105


### Step 2: Create a new dataframe called **recommendation_df**  that has film id and average of weighted ratings for that film 

In [46]:
recommendation_df = top_users_ratings.groupby("movieId").agg({"weighted_rating":"mean"})
recommendation_df.reset_index(inplace=True)
recommendation_df.head()

,movieId,weighted_rating
0,1,0.104256
1,2,0.153739
2,3,0.220866
3,4,0.026794
4,5,0.198394


### Step 3: Choose movies from **recommendation_df** that rating values are greater then 2 and sort movies up on weighted rating.

In [47]:
filtered_movies = recommendation_df[recommendation_df["weighted_rating"]>2]
filtered_movies.head()

,movieId,weighted_rating
7476,7823,2.183176
12039,55691,2.089951
12074,56006,2.487638
13761,70315,2.388515
15314,79866,3.178636


In [48]:
sorted_movies_to_recommend = filtered_movies.sort_values("weighted_rating",ascending=False)
sorted_movies_to_recommend.head()

,movieId,weighted_rating
15314,79866,3.178636
15393,80193,2.860772
21762,113434,2.634930
19782,102596,2.502162
12074,56006,2.487638


### Step 4: Get movie names from dataset and choose the top 5 movies to recommend

In [49]:
movies = pd.read_csv("./datasets/movie.csv")

In [50]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [51]:
print("*************************** THE FILMS THAT WILL BE RECOMMENDED ***************************")
print(sorted_movies_to_recommend.merge(movies,on="movieId").head()["title"])

*************************** THE FILMS THAT WILL BE RECOMMENDED ***************************
0              Schmatta: Rags to Riches to Rags (2009)
1                      No One Dies in Lily Dale (2010)
2                              What About Dick? (2012)
3    180° South (180 Degrees South) (180° South: Co...
4                    Ever Since the World Ended (2001)
Name: title, dtype: object


## ITEM BASED RECOMMENDER SYSTEM

### Step 1: Read datasets and get the current movie's id that has 5 star that chosen user gave.

In [52]:
movies = pd.read_csv("./datasets/movie.csv")
ratings = pd.read_csv("./datasets/rating.csv")

In [54]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [55]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [56]:
five_star_films = ratings[(ratings["userId"] == user_id) & (ratings["rating"] == 5)]
five_star_films.head()

,userId,movieId,rating,timestamp
18361123,127072,356,5.0,1999-12-21 12:54:06
18361124,127072,539,5.0,1999-12-21 12:54:36
18361129,127072,1517,5.0,1999-12-21 12:54:36
18361130,127072,1569,5.0,1999-12-21 12:48:01
18361133,127072,1777,5.0,1999-12-21 12:55:53


In [57]:
up_to_date_films = five_star_films.sort_values(by="timestamp", ascending=False)
up_to_date_films.head()

,userId,movieId,rating,timestamp
18361133,127072,1777,5.0,1999-12-21 12:55:53
18361124,127072,539,5.0,1999-12-21 12:54:36
18361129,127072,1517,5.0,1999-12-21 12:54:36
18361138,127072,2424,5.0,1999-12-21 12:54:36
18361123,127072,356,5.0,1999-12-21 12:54:06


In [65]:
movie_id = up_to_date_films[0:1]["movieId"].values[0]
movie_id

1777

### Step 2: Filter user_movie_df on "movie_id" that we have chosen.

In [60]:
user_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),"American President, The (1995)",Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Bad Boys (1995),Basic Instinct (1992),Batman (1989),Batman Begins (2005),Batman Forever (1995),Batman Returns (1992),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),Best in Show (2000),Beverly Hills Cop (1984),Beverly Hills Cop III (1994),Big (1988),Big Fish (2003),"Big Lebowski, The (1998)","Birdcage, The (1996)","Birds, The (1963)",Black Hawk Down (2001),Blade (1998),Blade Runner (1982),"Blair Witch Project, The (1999)",Blazing Saddles (1974),"Blues Brothers, The (1980)",Boogie Nights (1997),"Boot, Das (Boat, The) (1981)","Bourne Identity, The (2002)","Bourne Supremacy, The (2004)","Bourne Ultimatum, The (2007)",Bowling for Columbine (2002),Braveheart (1995),Brazil (1985),"Breakfast Club, The (1985)","Bridge on the River Kwai, The (1957)",Bridget Jones's Diary (2001),Broken Arrow (1996),"Bug's Life, A (1998)",Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Casablanca (1942),Casino (1995),Casino Royale (2006),Casper (1995),Cast Away (2000),Catch Me If You Can (2002),Charlie's Angels (2000),Chasing Amy (1997),Chicken Run (2000),Chinatown (1974),Chocolat (2000),"Christmas Story, A (1983)",Citizen Kane (1941),City Slickers II: The Legend of Curly's Gold (1994),City of God (Cidade de Deus) (2002),Clear and Present Danger (1994),Clerks (1994),"Client, The (1994)",Cliffhanger (1993),"Clockwork Orange, A (1971)",Close Encounters of the Third Kind (1977),Clueless (1995),Cocoon (1985),Con Air (1997),Coneheads (1993),Congo (1995),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Crimson Tide (1995),Crocodile Dundee (1986),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)","Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dark City (1998),"Dark Knight, The (2008)",Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),Deep Impact (1998),"Deer Hunter, The (1978)",Demolition Man (1993),"Departed, The (2006)",Desperado (1995),"Devil's Advocate, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Dirty Dancing (1987),Disclosure (1994),Dogma (1999),Donnie Brasco (1997),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dracula (Bram Stoker's Dracula) (1992),Dragonheart (1996),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),Edward Scissorhands (1990),Election (1999),Enemy of the State (1998),"English Patient, The (1996)",Eraser (1996),Erin Brockovich (2000),Eternal Sunshine of the Spotless Mind (2004),Executive Decision (1996),"Exorcist, The (1973)",Eyes Wide Shut (1999),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Ferris Bueller's Day Off (1986),"Few Good Men, A (1992)",Field of Dreams (1989),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)",First Knight (1995),"Fish Called Wanda, A (1988)"

In [69]:
movie_name = movies[movies["movieId"]==movie_id]["title"].values[0]
movie_name


'Wedding Singer, The (1998)'

In [71]:
chosen_movie_df = user_movie_df[movie_name]

### Step 3: Using the filtered dataframe find correlation with the selected movie and other movies

In [74]:
movies_to_recommend = user_movie_df.corrwith(chosen_movie_df).sort_values(ascending=False)

### Step 4: Recommend top 5 movies to recommend

In [87]:
movies_to_recommend = movies_to_recommend.reset_index()
movies_to_recommend

,title,0
0,"Wedding Singer, The (1998)",1.000000
1,Happy Gilmore (1996),0.475830
2,Liar Liar (1997),0.373356
3,American Pie (1999),0.354508
4,Meet the Parents (2000),0.347920
...,...,...
457,"Crying Game, The (1992)",0.004368
458,"Boot, Das (Boat, The) (1981)",0.000819
459,Dr. Strangelove or: How I Learned to Stop Worr...,-0.004694
460,Eyes Wide Shut (1999),-0.009334


In [90]:
print("*************************** THE FILMS THAT WILL BE RECOMMENDED ***************************")
movies_to_recommend[movies_to_recommend["title"] !=  movie_name]["title"].head()

*************************** THE FILMS THAT WILL BE RECOMMENDED ***************************


1       Happy Gilmore (1996)
2           Liar Liar (1997)
3        American Pie (1999)
4    Meet the Parents (2000)
5           Tommy Boy (1995)
Name: title, dtype: object